<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_1_Spring_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ===============================
# Bài 1: Hệ phần tử lò xo
# ===============================

import numpy as np

# Kết nối giữa các nút cho từng phần tử
element_nodes = np.array([
    [1, 2],
    [2, 3],
    [2, 4]
], dtype=int)

number_elements = element_nodes.shape[0]
number_nodes = 4

# Khởi tạo vectơ chuyển vị, vectơ lực và ma trận độ cứng
displacements = np.zeros((number_nodes, 1), dtype=float)
force = np.zeros((number_nodes, 1), dtype=float)
stiffness = np.zeros((number_nodes, number_nodes), dtype=float)

# Tải trọng tác dụng tại nút 2
# Lưu ý Python đánh chỉ số từ 0
force[1, 0] = 10.0

# Ma trận độ cứng phần tử (cục bộ)
K_local = np.array([[ 1.0, -1.0],
                    [-1.0,  1.0]])

# Lắp ghép Ma trận độ cứng tổng thể
for e in range(number_elements):
    dof = element_nodes[e] - 1
    i, j = dof[0], dof[1]
    stiffness[np.ix_([i, j], [i, j])] += K_local

# Các bậc tự do 1, 3, 4 cố định
# Lưu ý Python đánh chỉ số từ 0
fixed_dof = np.array([0, 2, 3])

all_dof = np.arange(number_nodes)

# active_dof là các bậc tự do có thể di chuyển
active_dof = np.setdiff1d(all_dof, fixed_dof)

# Giải hệ phương trình chỉ xét active_dof
K_ff = stiffness[np.ix_(active_dof, active_dof)]
f_f  = force[active_dof, :]
u_f = np.linalg.solve(K_ff, f_f)

# Cập nhật kết quả vào vectơ chuyển vị tổng thể
u = np.zeros((number_nodes, 1))
u[active_dof, :] = u_f

# Tính phản lực liên kết
reactions = stiffness @ u - force

# ===== Kết quả đầu ra =====
print("Global Stiffness Matrix K (Ma trận độ cứng tổng thể):\n", stiffness)
print("\nForce vector f (Vectơ lực):\n", force)
print("\nDisplacement vector u (Vectơ chuyển vị):\n", u)

print("\nReactions (Phản lực = K*u - f):\n", reactions)

# In phản lực tại các nút cố định
print("\nReactions at fixed nodes (Phản lực tại các nút cố định):")
for nd in (fixed_dof + 1):  # Chuyển về chỉ số bắt đầu từ 1 để hiển thị
    print(f"  Node {nd}: R = {reactions[nd-1,0]:.6g}")


Global Stiffness Matrix K (Ma trận độ cứng tổng thể):
 [[ 1. -1.  0.  0.]
 [-1.  3. -1. -1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  0.  1.]]

Force vector f (Vectơ lực):
 [[ 0.]
 [10.]
 [ 0.]
 [ 0.]]

Displacement vector u (Vectơ chuyển vị):
 [[0.        ]
 [3.33333333]
 [0.        ]
 [0.        ]]

Reactions (Phản lực = K*u - f):
 [[-3.33333333]
 [ 0.        ]
 [-3.33333333]
 [-3.33333333]]

Reactions at fixed nodes (Phản lực tại các nút cố định):
  Node 1: R = -3.33333
  Node 3: R = -3.33333
  Node 4: R = -3.33333


# New Section